# Homework 4

### CS328 — Numerical Methods for Visual Computing
- - -

**Out** on Friday 17.11, **due** on Friday 1.12.

This notebook contains literate code, i.e. brief fragments of Python surrounded by descriptive text. Please use the same format when submitting your answers. Begin your response to each problem with a <tt>&nbsp;<b>## Solution</b>&nbsp;&nbsp;</tt> markdown cell. Since this exercise includes a number of supplementary discussions, questions are explicitly marked with a **TODO** marker.

<br><div class="alert alert-warning">
Please keep in mind that homework assignments must be done individually.
</div>

# Installing additional prerequisites

In this assignment, we will make use a library called **[bqplot](https://github.com/bloomberg/bqplot)** developed by Bloomberg, which enables fully interactive plots within the Jupyter notebook. By default, this library is not installed on the Anaconda Python distribution we're using in this course, hence you will need to install it first. To do so, open the console (``cmd.exe`` on Windows or the standard terminal on Linux or MacOS) and enter the following commands.

```
pip install bqplot
jupyter nbextension enable --py --sys-prefix bqplot
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

**However**, note that some systems like MacOS ship a with default version of Python that is very old and outdated. To ensure that the right versions of the ``pip`` and ``jupyter`` commands are executed, it's safer to first have to navigate to the directory where Anaconda was installed and then execute the commands there. The following screenshots show how to do this on the various supported platforms (note the ``./`` prefix on Linux and Mac OS).

<a href="//rgl.s3.eu-central-1.amazonaws.com/media/uploads/wjakob/2016/12/05/bqplot.jpg"><img width="1000" src="//rgl.s3.eu-central-1.amazonaws.com/media/uploads/wjakob/2016/12/05/bqplot.jpg"></a>

Afterwards, you will have to **restart** Jupyter notebook for the change to become effective. You can enter and run the following commands in a new cell to check if bqplot was installed correctly—they should display a figure with a pie chart.

```python
import bqplot as bqp
bqp.Figure(marks=[bqp.Pie(sizes=range(1, 6))], title='A pie chart!')
```

In [1]:
import bqplot as bqp
bqp.Figure(marks=[bqp.Pie(sizes=range(1, 6))], title='A pie chart!')

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Pie(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales_metadata={'color': {'dimension': 'color'}}, sizes=array([1, 2, 3, 4, 5]), tooltip_style={'opacity': 0.9}, x=0.5, y=0.5)], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='A pie chart!')

# Prelude

We begin by importing essential NumPy/SciPy/Matplotlib components that are needed to complete the exercises. The package ``scipy.optimize`` is new -- it is only used in the last portion of this homework (hacker points). The ``ipywidgets`` package is  used internally by some of the code snippets provided by us.

In [2]:
import numpy as np
import scipy.linalg as la

# New: Optimization package
import scipy.optimize as opt

# bqplot plotting library
import bqplot as bqp

# Import graphical user interface components used below
from ipywidgets import interact
from ipywidgets import FloatSlider, VBox

# Inverse Kinematics using Newton's method and the Pseudoinverse
$$
\newcommand{\vp}{\mathbf{p}}
\newcommand{\vx}{\mathbf{x}}
\newcommand{\vf}{\mathbf{f}}
\newcommand{\mA}{\mathbf{A}}
$$
Lifelike computer-animated characters and animals are increasingly pervasive in today’s society:  they are now commonly encountered in games, advertisements, feature animation, and a variety of other fields. It’s interesting to realize that all of these animated characters initially start out as static 3D shapes, not unlike stone sculptures that are unable to move. Before a character built in this way can be seen in motion, the precise way in which its shape can change over time must be characterized mathematically. The most common way of doing this entails designing a customized 3D skeleton that is then attached to its outer skin. Subsequently, any adjustment to the posture of the skeleton will result in a corresponding change to the posture of the character. The figure below shows a simple character with an embedded skeleton and two example poses that were created by  rotating the joints of the skeleton.

<br><img width="490" src="//rgl.s3.eu-central-1.amazonaws.com/media/uploads/wjakob/2016/12/06/ik-system.jpg"><br>

A skeleton can range from a few elements to massively complex arrangements that reproduce the entire biological structure of the person or animal in question. In either case, the skeleton consists only of repeated instances of one basic component: a *bone*. Fortunately, computer graphics bones are much simpler than actual bones in real life.

<br><img width="490" src="//rgl.s3.eu-central-1.amazonaws.com/media/uploads/wjakob/2016/12/06/inverse-kinematics-05.png"><br>

As you can see in the above figure, each bone is essentially a line segment with a joint where it is connected to the previous bone. The bone can rotate in any way, but the connection between two bones can never move apart. Each bone also has a fixed length, in other words: it is rigid and never compresses or expands. On the other side, the next bone is generally attached. In 3D, a variety of rotations are possible (e.g. around the X, Y, or Z axis). In two dimensions things are simpler, and a single angle is enough to completely characterize the rotation of a bone relative to its predecessor bone. Everything in this homework will be in two dimensions to keep things simple.

## Part 1: Forward Kinematics (40 pts)

In this exercise, we'll investigate the mathematics of a very simple kind of "skeleton": a chain of bones in two dimensions with joint positions $\vp_i = (x_i, y_i)$. The first joint is rigidly attached to the origin (i.e. $\vp_0 = (0, 0)$) while the other joints and bones are free to move in any way. For simplicitly, we'll also assume that all of the bones have the same length $l_1=l_2=\ldots=1$.

<img width="490" src="//rgl.s3.eu-central-1.amazonaws.com/media/uploads/wjakob/2016/12/06/inverse-kinematics-01.png"> 

Each parameter $\theta_i\in[0,2\pi]$ specifies the counter-clockwise angle that the associated bone from joint $\vp_{i-1}$ to joint $\vp_i$ makes with its predecessor bone (the pair of bones are parallel if $\theta_i=0$). The first bone doesn't have a predecessor, hence $\theta_1$ is measured relative to the $X$ axis. Note how the complete set of bone angles $\theta_1, \theta_2, \ldots$ is all the information we need to compute the precise positions of all the joint positions in Euclidean space.

Forward kinematics (FK) is defined as the problem of converting a set of bone angles $\theta_i$ into joint positions $\vp_i$. Since $\vp_i$ depends on all of the preceding angles, we can think of each joint position as a function $\vp_i=\vp(\theta_1,\ldots,\theta_{i})$ 

**TODO** (15 pts): Your first task is to create a function ``chain_simple``, which solves the forward kinematics for a chain with at most one bone. The function should take an array of angles as a parameter, which can be of length 0 or 1 (use the Python ``len()`` function to query the length of an array). When no angles are specified, the function should return the position of the first joint $(x_0,y_0)=(0, 0)$ as an 1D NumPy array. When a single angle is specified, it should return the position $x_1, y_1$.

Ensure that your implementation satisfies the following equalities (up to minor rounding errors):

1. ``chain_simple([]) == np.array([0., 0.])`` 
2. ``chain_simple([0.]) == np.array([1., 0.])``.
3. ``chain_simple([np.pi / 4]) == np.array([ 0.70710678,  0.70710678])``.

## Solution

In [3]:
def chain_simple(theta):
    if len(theta) == 0:
        return np.array([0., 0.])
    elif len(theta) == 1:
        return np.array([np.cos(theta[0]), np.sin(theta[0])])
    else:
        print("Error: too many arguments!")
    
print("Tests for the function chain_simple\n***********************************")
print("1. ")
print(chain_simple([]))
print(np.array([0., 0.]))
print(chain_simple([]) == np.array([0., 0.]))
print("2. ")
print(chain_simple([0.]))
print(np.array([1., 0.]))
print(chain_simple([0.]) == np.array([1., 0.]))
print("3. ")
#print(chain_simple([np.pi / 4]))
print("[ %.12f %.12f]" % (tuple(chain_simple([np.pi / 4]))))
print(np.array([ 0.70710678,  0.70710678]))
print(chain_simple([np.pi / 4]) == np.array([ 0.70710678,  0.70710678]))
if np.testing.assert_array_almost_equal(chain_simple([np.pi / 4]), np.array([ 0.70710678,  0.70710678]), decimal = 9) == None:
    print("The arrays are equal up to precision 9")

Tests for the function chain_simple
***********************************
1. 
[ 0.  0.]
[ 0.  0.]
[ True  True]
2. 
[ 1.  0.]
[ 1.  0.]
[ True  True]
3. 
[ 0.707106781187 0.707106781187]
[ 0.70710678  0.70710678]
[False False]
The arrays are equal up to precision 9


# 1.1 Helper function

We provide the function ``fk_demo()`` below to interactively explore the possible chain configurations via forward kinematics. The implementation uses the ``bqplot`` library mentioned above and is fairly technical. You are welcomed but not expected to read or understand how it works.

In [4]:
def fk_demo(chain_func, theta, extra = [[],[]]):
    '''
    This function visualizes the configuration of a chain of bones
    and permits interactive changes to its state. It expects two arguments:
    
    ``chain_func``: a function that implements forward kinematics by
    turning a sequence of angles (theta_1, theta_2, ..., theta_n) into
    the position of the last joint of this chain (x_n, y_n).
    
    ``theta``: an array with the initial angles of all joints
    
    ``extra``: An optional argument which can be used to plot
    additional points that are highlighted in red
    '''
    
    # Function which repeatedly calls ``chain_func`` to compute all joint positions
    def chain_all(theta):
        return np.column_stack([chain_func(theta[:i]) for i in range(0, len(theta) + 1)])

    # Determine size and initial configuration
    size = len(theta)
    positions = chain_all(theta)

    # Define the range of the plotting frame
    scales = { 'x': bqp.LinearScale(min=-size-1, max=size+1),
               'y': bqp.LinearScale(min=-size-1, max=size+1) }

    # Create a scatter plot (for joints), a line plot (for bones), and
    # another scatter plot (to draw extra points specified the ``extra`` argument)
    scat  = bqp.Scatter(scales=scales)
    lines = bqp.Lines(scales=scales)
    scat2 = bqp.Scatter(scales=scales, default_colors=['red'])

    # Create a figure that combines the three plots
    figure = bqp.Figure(marks=[scat, scat2, lines])
    figure.layout.height = '500px'
    figure.layout.width = '500px'
    
    # Initialize the plots with the initial data
    scat.x, scat.y = positions
    lines.x, lines.y = positions
    scat2.x, scat2.y = extra
    
    sliders = []
    
    # For each angle theta_i,
    for i in range(len(theta)):
        # Create a graphical slider
        slider = FloatSlider(min=0, max=2*np.pi, value=theta[i], step=1e-3)
        
        # Define a callback function that will be triggered when the slider is moved
        def callback(value, i = i):
            theta[i] = value['new']
            positions = chain_all(theta)
            scat.x, scat.y = positions
            lines.x, lines.y = positions

        # "Attach" the callback function to the slider
        slider.observe(callback, 'value')
        sliders.append(slider)

    # Combine the plots and sliders in a vertical arrangement
    return VBox([*sliders, figure])

## 1.2 Visualization of the forward kinematics

**TODO (0pts)**: To ensure that your implementation of ``chain_simple`` satisfies all the specifications, invoke the ``fk_demo()`` function with arguments ``chain_simple`` and ``[0.]`` (the initial parameters of a flat chain). You should be able to drag a slider from 0 to $2\pi$ and see a visual representation of a 1-bone chain turning counter-clockwise.

## Solution 1.2

In [5]:
fk_demo(chain_simple, [0.])

VBox(children=(FloatSlider(value=0.0, max=6.283185307179586, step=0.001), Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', min_width='125px', width='500px'), marks=[Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=2.0, min=-2.0), 'y': LinearScale(max=2.0, min=-2.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.]), y=array([ 0.,  0.])), Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=2.0, min=-2.0), 'y': LinearScale(max=2.0, min=-2.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=2.0, min=-2.0), 'y': LinearScale(max=2.0, min=-2.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.]), y=array([ 0.,  0.]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))))

## 1.3 Longer chains

**TODO** (20 pts): Create a function ``chain``, which solves the forward kinematics for an arbitrarily long sequence of bones. The function should take an arbitrary-length array of angles as a parameter. When no angles are specified, the function should return the position $(x_0, y_0)$ as before. When $i$ angles are specified, it should (only) return  the joint position $(x_{i}, y_{i})$. You'll likely want to use recursion, which allows for a particularly simple implementation.

Ensure that your implementation satisfies the previously listed equalities in addition to the following two (up to minor rounding errors):

1. ``chain([0.1, 0.2, 0.3, 0.4]) == np.array([ 3.31597858,  1.80146708])``
2. ``chain([np.pi, np.pi, np.pi, np.pi]) == np.array([0, 0])``

## Solution 1.3

I have written two solutions, one recursive and one iterative for performance purposes. For the rest of the homweork, I'll still use the recursive implementation.

In [6]:
def chain(theta):
    if len(theta) <= 1:
        return chain_simple(theta)
    def chainRec(theta, thetaAcc):
        if len(theta) <= 1:         
            thetaAcc += theta[0]
            return np.array([np.cos(thetaAcc), np.sin(thetaAcc)])
        else:
            thetaAcc += theta[0]
            posJoint = chainRec(np.delete(theta, 0), thetaAcc)
            return posJoint + np.array([np.cos(thetaAcc), np.sin(thetaAcc)])
    #The initial value of thetaAcc is 0
    return chainRec(theta, 0)

def chainIter(theta):
    thetaAcc = theta[0]
    posJoint = chain_simple([theta[0]])
    for i in range(1, len(theta)):
        thetaAcc += theta[i]
        posJoint = posJoint + [np.cos(thetaAcc), np.sin(thetaAcc)]
    return posJoint

import time
tim = time.process_time()
pos = chain([0.1, 0.2, 0.3, 0.4])
#print("Time rec = ", time.process_time()-tim)
print("1. ", pos)
tim = time.process_time()
posIter = chainIter([0.1, 0.2, 0.3, 0.4])
#print("Time iter = ", time.process_time()-tim)
#print(posIter)
if np.testing.assert_array_almost_equal(pos, np.array([ 3.31597858,  1.80146708]), decimal = 8) == None:
    print("The arrays are equal up to precision 8") 
pos = chain([np.pi, np.pi, np.pi, np.pi])
print("2. ", pos) 
posRef = np.array([0, 0]) 
if np.abs(pos[0] - posRef[0]) < 1.e-9 and np.abs(pos[1]- posRef[1]) < 1.e-9:
    print("The arrays are equal up to precision 9") 

1.  [ 3.31597858  1.80146708]
The arrays are equal up to precision 8
2.  [  0.00000000e+00  -2.44929360e-16]
The arrays are equal up to precision 9


## 1.4 Attempting to reach a certain position

Run the command ``fk_demo(chain, [0, 0, 0, 0, 0], [[-2], [3]])`` below. You should see a chain with five segments and five corresponding sliders, as well as an additional point highlighted in red.

**TODO (5 points)**: Find a configuration of angles that brings the endpoint of the chain as close as possible to the highlighted location ``[-2, 3]]``. An exact match is not necessary, but the points should overlap by a significant margin. Copy the parameters you found into the argment list of the ``fk_demo`` function call.

## Solution 1.4


In [7]:
fk_demo(chain, [1.55, 0.38, 0.12, 0.03, 1.84], [[-2], [3]],)

VBox(children=(FloatSlider(value=1.55, max=6.283185307179586, step=0.001), FloatSlider(value=0.38, max=6.283185307179586, step=0.001), FloatSlider(value=0.12, max=6.283185307179586, step=0.001), FloatSlider(value=0.03, max=6.283185307179586, step=0.001), FloatSlider(value=1.84, max=6.283185307179586, step=0.001), Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', min_width='125px', width='500px'), marks=[Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.        ,  0.02079483, -0.33073401, -0.79180671, -1.27928881,
       -1.99132152]), y=array([ 0.        ,  0.99978376,  1.93596082,  2.82332319,  3.69645616,
        2.99430988])), Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([-2]), y=array([3])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.        ,  0.02079483, -0.33073401, -0.79180671, -1.27928881,
       -1.99132152]), y=array([ 0.        ,  0.99978376,  1.93596082,  2.82332319,  3.69645616,
        2.99430988]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))))

We found, for exemple, this suitable configuration:
[1.55, 0.38, 0.12, 0.03, 1.84]

# Part 2: Inverse Kinematics (60 pts)

Problems similar to the one in Section 1.4 are tedious to solve by hand: all of the parameters are interdependent and must be adjusted in a coordinated manner. So-called *inverse kinematics* techniques apply numerical root finding to determine solutions to this problem in an automated way. Most modern animation systems have builtin support for inverse kinematics since it allows for a much more convenient workflow: rather than having to tweak each individual bone, artists can directly specify a target shape, and the system will automatically infer all the necessary rotations.

In this part of the exercise, we will use inverse kinematics to automatically determine $\theta_1,\ldots,\theta_n$ such that

$$
\vp(\theta_1,\ldots,\theta_n) = \vp_{\mathrm{target}}
$$

for a given value $\vp_{\mathrm{target}}\in\mathbb{R}^2$. In other words: the user can move around the endpoint of the chain, and the skeleton will automatically reconfigure itself to follow. This is illustrated in the following figure:

<img width="900" src="//rgl.s3.eu-central-1.amazonaws.com/media/uploads/wjakob/2016/12/06/inverse-kinematics-04.png"> 

All good numerical root finding techniques require the ability to evaluate the Jacobian of $\vp$, i.e. all the partial derivatives $\frac{\partial\vp(\theta_1,\ldots,\theta_n)}{\partial \theta_j}$. The partial derivatives encode how a small perturbation of each of the angles $\theta_j$ leads to a corresponding change in $\vp(\theta_1,\ldots,\theta_n)$. As before, we'll first look at a 1-segment chain and then derive a solution for the general problem.

**TODO** (10 pts): Implement a function ``dchain_simple(theta)`` which takes an array with one entry, and which computes the function $\frac{\partial \vp(\theta_1)}{\partial \theta_1}$. The return value should be a two-dimensional array with one column and two rows containing the partial derivatives of the coordinate values $x_1$ and $y_1$. You should use analytic methods -- approximating the derivatives via finite differences is not allowed.

Ensure that your implementation satisfies the following equalities (up to minor rounding errors):

1. ``dchain_simple([0]) == np.array([[ 0.], [ 1.]])``.

   In other words: a small perturbation around the angle $\theta_1=0$ leads to a corresponding change in the $y_1$ coordinate.<br><br>
   
3. ``dchain_simple([np.pi / 4]) == np.array([[-0.70710678], [ 0.70710678]])``.

## Solution 2

In [8]:
def dchain_simple(theta):
    if len(theta) == 0:
        return np.array([[0.], [1.]])
    elif len(theta) == 1:
        return np.array([[-np.sin(theta[0])],[np.cos(theta[0])]])
    else:
        print("Error: too many arguments!")

print("1. ")
print(dchain_simple([0]))
print(np.array([[ 0.], [ 1.]]))
print(dchain_simple([0]) == np.array([[ 0.], [ 1.]]))

print("2. ")
#print(dchain_simple([np.pi / 4]))
print("[[%.12f] \n [%.12f]]" % (tuple(dchain_simple([np.pi / 4]))))
print(np.array([[-0.70710678], [ 0.70710678]]))
print(dchain_simple([np.pi / 4]) == np.array([[-0.70710678], [ 0.70710678]]))
if np.testing.assert_array_almost_equal(dchain_simple([np.pi / 4]), np.array([[-0.70710678], [ 0.70710678]]), decimal = 9) == None:
    print("The arrays are equal up to precision 9")

1. 
[[-0.]
 [ 1.]]
[[ 0.]
 [ 1.]]
[[ True]
 [ True]]
2. 
[[-0.707106781187] 
 [0.707106781187]]
[[-0.70710678]
 [ 0.70710678]]
[[False]
 [False]]
The arrays are equal up to precision 9


## 2.1 Implementing the full Jacobian function

Having finished the version for a single bone, we'll now turn to the full Jacobian $\nabla \vp(\theta_1, \ldots, \theta_n)$, which is a $2\times n$ matrix containing the partial derivatives with respect to all angles. You'll likely want to use a vector version of the [product](https://en.wikipedia.org/wiki/Product_rule) or [chain rule](https://en.wikipedia.org/wiki/Chain_rule) in your your implementation. Specifically, note that

$$
\frac{\partial}{\partial t} \left[\mA(t)\vx(t)\right] = \mA'(t)\vx(t) + \mA(t)\vx'(t)
$$

where $\mA(t)$ and $\vx(t)$ are a matrix and a vector depending on a parameter $t$, respectively.

**TODO** (30 pts): Implement a function ``dchain(theta)`` which accepts an 1D array of angles with length $\ge 1$ and computes the Jacobian $\nabla \vp(\theta_1, \ldots, \theta_n)$, a $2\times n$ matrix.

Ensure that your implementation satisfies the following equalities (up to minor rounding errors):

1. ``dchain([0, 0, 0, 0]) == np.array([[ 0.,  0.,  0.,  0.], [ 4.,  3.,  2.,  1.]])``.

   In other words: for a length-4 chain, the endpoint moves the most when the first joint is perturbed, while later joints have less of an effect.<br><br>
   
3. ``dchain([0.1, 0.2, 0.3]) == np.array([[-0.9599961 , -0.86016268, -0.56464247], [ 2.77567627,  1.7806721 ,  0.82533561]])``.

## Solution 2.1

In [9]:
def dchain(theta):
    if len(theta) == 1:
        return dchain_simple(theta)   
    def dchainRec(theta, acc, flipJacobian):
        if len(theta) == 0:
            return np.fliplr(flipJacobian)
        else:
            acc = acc + dchain_simple(np.array([np.sum(theta)]))
            flipJacobian = np.concatenate((flipJacobian, acc), axis = 1)
            return dchainRec(np.delete(theta,len(theta)-1), acc, flipJacobian)
    # The initial values for acc and flipJacobian are, respectively, np.array([[0.], [0.]] and np.array([[],[]])
    return dchainRec(theta, np.array([[0.], [0.]]), np.array([[],[]]))

print("1. ")
print(dchain([0, 0, 0, 0]))
print(np.array([[ 0.,  0.,  0.,  0.], [ 4.,  3.,  2.,  1.]]))
print(dchain([0, 0, 0, 0]) == np.array([[ 0.,  0.,  0.,  0.], [ 4.,  3.,  2.,  1.]]))

print("2. ")
print(dchain([0.1, 0.2, 0.3]))
print(np.array([[-0.9599961 , -0.86016268, -0.56464247], [ 2.77567627,  1.7806721 ,  0.82533561]]))
print(dchain([0.1, 0.2, 0.3]) == np.array([[-0.9599961 , -0.86016268, -0.56464247], [ 2.77567627,  1.7806721 ,  0.82533561]]))
if np.testing.assert_array_almost_equal(dchain([0.1, 0.2, 0.3]), np.array([[-0.9599961 , -0.86016268, -0.56464247], [ 2.77567627,  1.7806721 ,  0.82533561]]), decimal = 8) == None:
    print("The arrays are equal up to precision 8")


1. 
[[ 0.  0.  0.  0.]
 [ 4.  3.  2.  1.]]
[[ 0.  0.  0.  0.]
 [ 4.  3.  2.  1.]]
[[ True  True  True  True]
 [ True  True  True  True]]
2. 
[[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]
[[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]
[[False False False]
 [False False False]]
The arrays are equal up to precision 8


## 2.2 Solving the inverse kinematics problem using Newton's Method

Newton's method is one of the most widely used methods for finding solutions to systems of non-linear equations. It  converges at a remarkable speed when started sufficiently close to a root, though there is generally no strict guarantee of convergence.

Given a function $\vf(\vx)$, Newton's method tries to find a solution to the equation $\vf = 0$ using steps of the form

$$
\vx_{i+1}=\vx_i - \left(\nabla \vf\right)^{-1}\vf(\vx_{i}).
$$

In the context of inverse kinematics, we want to apply Newton's method to solve an equation of the form

$$
\vp(\theta_1,\ldots,\theta_n) = \vp_{\mathrm{target}}.
$$

for a given reference position $\vp_{\mathrm{target}}\in\mathbb{R}^2$.

In other words: the unknowns are the angles $\theta_1,\ldots,\theta_n$, and the function whose root we seek maps to a two-dimensional domain. It is not immediately obvious how to apply Newton's method, since the Jacobian of the function has the shape $2\times n$ and hence cannot be inverted using standard techniques like the LU decomposition.

This should not be surprising. It is a consequence of the fact that many different configurations can be used to reach the same $\vp_{\mathrm{target}}$, which you may have noticed in part 1.4.

Fortunately, we can use the *pseudoinverse*, a generalization of the inverse to non-square matrices. In this specific case, the Jacobian is *wide* (i.e. it has more columns than rows), in which case the pseudoinverse will find the solution to a linear system which has the smallest $\|\cdot\|_2$-norm. That is excellent news, since it causes the IK solver to make small adjustments to the angles to reach a new position.

**TODO** (15 pts): Implement a function ``newton(theta, target)`` that takes a 1-dimensional array of angles as a starting guess as well as a 2D target position (also specified as a 1-dimensional array) as input. The implementation should perform a fixed 8 iterations of Newton's method to try to solve the equation $\vp(\theta_1,\ldots,\theta_n) = \vp_{\mathrm{target}}$ and return the final set of parameters $\theta_1,\ldots,\theta_n$ as an 1-dimensional NumPy array. You can use the function ``la.pinv`` to compute the pseudoinverse.

Ensure that your implementation is able to converge to the following positions (up to minor rounding errors)

1. Moving a 1-element chain from the default configuration to position $(0, 1)$, i.e. ``chain(newton(np.array([0.]), np.array([0., 1.]))) == np.array([0, 1])``<br><br>

1. Moving a 2-element chain from the default configuration to position $(0.5, 0.5)$, i.e. ``chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))) == np.array([0.5, 0.5])``

## Solution 2.2

In [10]:
def newton(theta, target):
    #The implementation performs a fixed 8 iterations of Newton's method
    for i in range(8):
        pos = chain(theta)
        deltaPos = pos - target
        jacobian = dchain(theta)
        pseudoInvJacobian = la.pinv(jacobian)
        deltaTheta = (pseudoInvJacobian @ deltaPos[:, np.newaxis]).squeeze()
        theta = theta - deltaTheta    
    return theta

print("1. ")
print(chain(newton(np.array([0.]), np.array([0., 1.]))))
print(np.array([0, 1]))
print(chain(newton(np.array([0.]), np.array([0., 1.]))) == np.array([0, 1]))
if np.testing.assert_array_almost_equal(chain(newton(np.array([0.]), np.array([0., 1.]))), np.array([0, 1]), decimal = 16) == None:
    print("The arrays are equal up to precision 16")


print("2. ")
#print(chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))))
print("[ %.16f %.16f]" % (tuple(chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))))))
print(np.array([0.5, 0.5]))
print(chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))) == np.array([0.5, 0.5]))
if np.testing.assert_array_almost_equal(chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))), np.array([0.5, 0.5]), decimal = 15) == None:
    print("The arrays are equals up to precision 15")

1. 
[  6.12323400e-17   1.00000000e+00]
[0 1]
[False  True]
The arrays are equal up to precision 16
2. 
[ 0.4999999999999988 0.4999999999999996]
[ 0.5  0.5]
[False False]
The arrays are equals up to precision 15


## 2.3 One more helper function

We provide the function ``ik_demo()`` below to interactively explore the possible chain configurations via inverse kinematics. Similar to ``fk_demo()``, the function is fairly technical. You are welcomed but not expected to read or understand how it works.

I made a litle change in this method in order to plot a red endpoint.

In [11]:
def ik_demo(solver, size):    
    theta = np.zeros(size, dtype=np.float64)
    
    # Function which repeatedly calls ``chain`` to compute all joint positions
    def chain_all(theta):
        return np.column_stack([chain(theta[:i]) for i in range(0, len(theta) + 1)])

    # Callback that is invoked when the user drags the red endpoint around
    def refresh(_):
        # 'theta' is a variable of the parent function, we want to modify it here
        nonlocal theta
        
        # Target position
        target = np.array([scat2.x[0], scat2.y[0]])
        
        # Don't try to solve the problem if the user dragged the point out of the circle
        if la.norm(target) > size:
            return
        
        # Call the provided IK solver
        theta = solver(theta, target)
        
        # Update the positions
        values = chain_all(theta)
        scat.x, scat.y = values
        lines.x, lines.y = values
    
    # Similar to fk_solver(), create a number of plots and merge them
    scales = { 'x': bqp.LinearScale(min=-size-1, max=size+1),
               'y': bqp.LinearScale(min=-size-1, max=size+1) }

    scat  = bqp.Scatter(scales=scales)
    lines = bqp.Lines(scales=scales)

    # Create a circle which marks the boundary of where the red point can be moved
    circle_x = np.cos(np.linspace(0, 2*np.pi, 100)) * size
    circle_y = np.sin(np.linspace(0, 2*np.pi, 100)) * size
    circle = bqp.Lines(x=circle_x, y=circle_y,
                       scales=scales, colors=['gray'])
    
    # Special plot, which contains the red endpoint that can be moved
    scat2 = bqp.Scatter(scales=scales,
                        enable_move=True, 
                        update_on_move=True,
                        #default_colors=['red'])
                        colors=['red'])

    # Initialize the visualizations with the default configuration
    values = chain_all(theta)
    scat.x, scat.y = values
    lines.x, lines.y = values
    scat2.x, scat2.y = chain(theta).reshape(2, 1)
    
    # Call the 'refresh' function when the red dot is moved
    scat2.observe(refresh, names=['x', 'y'])

    figure = bqp.Figure(marks=[scat, scat2, lines, circle])
    figure.layout.height = '500px'
    figure.layout.width = '500px'
    return figure

## 2.4 Putting everything together

Finally, let's visualize the behavior of the completed inverse kinematics solver. 

**TODO** (5 pts):
1. Invoke the IK demonstration with with 4 segments, i.e. ``ik_demo(newton, 4)``. You should be able to move the red endpoint with your mouse cursor, leading to a smooth adjustment of the chain configuration.<br><br>

2. Invoke the IK demonstration with with 30 segments, i.e. ``ik_demo(newton, 30)``. Does the algorithm still work? Can you break it by moving the cursor too quickly? What happens in this case? Can you recover?

## Solution 2.4.1

In [12]:
ik_demo(newton, 4)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', min_width='125px', width='500px'), marks=[Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=5.0, min=-5.0), 'y': LinearScale(max=5.0, min=-5.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.,  2.,  3.,  4.]), y=array([ 0.,  0.,  0.,  0.,  0.])), Scatter(colors=['red'], enable_move=True, interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=5.0, min=-5.0), 'y': LinearScale(max=5.0, min=-5.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, update_on_move=True, x=array([ 4.]), y=array([ 0.])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=5.0, min=-5.0), 'y': LinearScale(max=5.0, min=-5.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.,  2.,  3.,  4.]), y=array([ 0.,  0.,  0.,  0.,  0.])), Lines(colors=['gray'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=5.0, min=-5.0), 'y': LinearScale(max=5.0, min=-5.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 4.        ,  3.99194671,  3.96781925,  3.92771479,  3.87179481,
        3.80028447,  3.71347173,  3.61170615,  3.49539751,  3.36501413,
        3.22108103,  3.06417777,  2.89493615,  2.71403765,  2.52221067,
        2.32022764,  2.10890187,  1.8890843 ,  1.66166005,  1.42754489,
        1.1876815 ,  0.94303574,  0.69459271,  0.4433528 ,  0.19032766,
       -0.06346386, -0.31699983, -0.56925935, -0.81922667, -1.06589525,
       -1.30827185, -1.5453805 , -1.77626645, -2.        , -2.21568026,
       -2.42243875, -2.61944294, -2.80589955, -2.9810578 , -3.14421238,
       -3.29470633, -3.43193365, -3.55534179, -3.66443383, -3.75877048,
       -3.83797189, -3.90171915, -3.94975555, -3.98188769, -3.99798617,
       -3.99798617, -3.98188769, -3.94975555, -3.90171915, -3.83797189,
       -3.75877048, -3.66443383, -3.55534179, -3.43193365, -3.29470633,
       -3.14421238, -2.9810578 , -2.80589955, -2.61944294, -2.42243875,
       -2.21568026, -2.        , -1.77626645, -1.5453805 , -1.30827185,
       -1.06589525, -0.81922667, -0.56925935, -0.31699983, -0.06346386,
        0.19032766,  0.4433528 ,  0.69459271,  0.94303574,  1.1876815 ,
        1.42754489,  1.66166005,  1.8890843 ,  2.10890187,  2.32022764,
        2.52221067,  2.71403765,  2.89493615,  3.06417777,  3.22108103,
        3.36501413,  3.49539751,  3.61170615,  3.71347173,  3.80028447,
        3.87179481,  3.92771479,  3.96781925,  3.99194671,  4.        ]), y=array([  0.00000000e+00,   2.53695679e-01,   5.06369814e-01,
         7.57004977e-01,   1.00459195e+00,   1.24813378e+00,
         1.48664982e+00,   1.71917965e+00,   1.94478694e+00,
         2.16256327e+00,   2.37163172e+00,   2.57115044e+00,
         2.76031605e+00,   2.93836683e+00,   3.10458586e+00,
         3.25830381e+00,   3.39890172e+00,   3.52581345e+00,
         3.63852798e+00,   3.73659144e+00,   3.81960897e+00,
         3.88724627e+00,   3.93923101e+00,   3.97535386e+00

## Solution 2.4.2

In [13]:
ik_demo(newton, 30)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', min_width='125px', width='500px'), marks=[Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=31.0, min=-31.0), 'y': LinearScale(max=31.0, min=-31.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.]), y=array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])), Scatter(colors=['red'], enable_move=True, interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=31.0, min=-31.0), 'y': LinearScale(max=31.0, min=-31.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, update_on_move=True, x=array([ 30.]), y=array([ 0.])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=31.0, min=-31.0), 'y': LinearScale(max=31.0, min=-31.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.]), y=array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])), Lines(colors=['gray'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=31.0, min=-31.0), 'y': LinearScale(max=31.0, min=-31.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 30.        ,  29.93960029,  29.75864438,  29.45786092,
        29.03846104,  28.50213353,  27.85103799,  27.08779615,
        26.21548131,  25.23760598,  24.15810773,  22.98133329,
        21.71202114,  20.35528235,  18.91658001,  17.40170729,
        15.81676403,  14.16813224,  12.46245039,  10.70658665,
         8.90761126,   7.07276807,   5.20944533,   3.325146  ,
         1.42745747,  -0.47597892,  -2.37749871,  -4.26944515,
        -6.14420004,  -7.99421441,  -9.8120389 , -11.59035377,
       -13.32199838, -15.        , -16.61760192, -18.16829061,
       -19.64582202, -21.04424663, -22.35793349, -23.58159284,
       -24.71029744, -25.7395024 , -26.66506346, -27.48325372,
       -28.19077862, -28.78478921, -29.26289361, -29.62316666,
       -29.86415768, -29.98489627, -29.98489627, -29.86415768,
       -29.62316666, -29.26289361, -28.78478921, -28.19077862,
       -27.48325372, -26.66506346, -25.7395024 , -24.71029744,
       -23.58159284, -22.35793349, -21.04424663, -19.64582202,
       -18.16829061, -16.61760192, -15.        , -13.32199838,
       -11.59035377,  -9.8120389 ,  -7.99421441,  -6.14420004,
        -4.26944515,  -2.37749871,  -0.47597892,   1.42745747,
         3.325146  ,   5.20944533,   7.07276807,   8.90761126,
        10.7065

Yes, the algorithm still works.
No, I couldn not (except once when I could't recover).

# 3 Hacker points (10 points, optional)

The inverse kinematics solution from Example 2 provides generally reasonable results by attempting to make the smallest change to the bone angles that will allow the chain to reach a particular point. However, in some cases there might be additional requirements we'd like the solution to satisfy. For instance, it might be unnatural for the character to bend a joint more than a few degrees. In this case, we could try to find a solution to an optimization problem that compromises between reaching the target position and bending joints by an overly large amount.

$$
\DeclareMathOperator*{\argmin}{argmin}
\argmin_{\theta_1,\ldots,\theta_n} \alpha\cdot\|\vp(\theta_1,\ldots,\theta_n)-\vp_\mathrm{target}\|_2^2 + \sum_{i=1}^n\theta_i^2
$$

**TODO**: Optimize the above objective using the function ``opt.minimize`` and run your algorithm on a chain of length 5 (using the ``ik_demo`` function). Use $\alpha=10$ in the above formula (i.e. reaching the target point is quite important).

## Solution 3.1

In [14]:
def solverOpti1(th, target):
    alpha = 10
    def f_obj(theta):
        return alpha*la.norm(chain(theta) - target)**2 + np.math.fsum(np.square(theta))
    res = opt.minimize(f_obj, th)
    return res.x

ik_demo(solverOpti1, 5)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', min_width='125px', width='500px'), marks=[Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.,  2.,  3.,  4.,  5.]), y=array([ 0.,  0.,  0.,  0.,  0.,  0.])), Scatter(colors=['red'], enable_move=True, interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, update_on_move=True, x=array([ 5.]), y=array([ 0.])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.,  2.,  3.,  4.,  5.]), y=array([ 0.,  0.,  0.,  0.,  0.,  0.])), Lines(colors=['gray'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 5.        ,  4.98993338,  4.95977406,  4.90964349,  4.83974351,
        4.75035559,  4.64183967,  4.51463269,  4.36924689,  4.20626766,
        4.02635129,  3.83022222,  3.61867019,  3.39254706,  3.15276334,
        2.90028455,  2.63612734,  2.36135537,  2.07707507,  1.78443111,
        1.48460188,  1.17879468,  0.86824089,  0.554191  ,  0.23790958,
       -0.07932982, -0.39624978, -0.71157419, -1.02403334, -1.33236907,
       -1.63533982, -1.93172563, -2.22033306, -2.5       , -2.76960032,
       -3.02804844, -3.27430367, -3.50737444, -3.72632225, -3.93026547,
       -4.11838291, -4.28991707, -4.44417724, -4.58054229, -4.6984631 ,
       -4.79746487, -4.87714893, -4.93719444, -4.97735961, -4.99748271,
       -4.99748271, -4.97735961, -4.93719444, -4.87714893, -4.79746487,
       -4.6984631 , -4.58054229, -4.44417724, -4.28991707, -4.11838291,
       -3.93026547, -3.72632225, -3.50737444, -3.27430367, -3.02804844,
       -2.76960032, -2.5       , -2.22033306, -1.93172563, -1.63533982,
       -1.33236907, -1.02403334, -0.71157419, -0.39624978, -0.07932982,
        0.23790958,  0.554191  ,  0.86824089,  1.17879468,  1.48460188,
        1.78443111,  2.07707507,  2.36135537,  2.63612734,  2.90028455,
        3.15276334,  3.39254706,  3.61867019,  3.83022222,  4.02635129,
        4.20626766,  4.36924689,  4.51463269,  4.64183967,  4.75035559,
        4.83974351,  4.90964349,  4.95977406,  4.98993338,  5.        ]), y=array([  0.00000000e+00,   3.17119598e-01,   6.32962268e-01,
         9.46256222e-01,   1.25573994e+00,   1.56016723e+00,
         1.85831228e+00,   2.14897456e+00,   2.43098368e+00,
         2.70320409e+00,   2.96453965e+00,   3.21393805e+00,
         3.45039506e+00,   3.67295854e+00,   3.88073232e+00,
         4.07287976e+00,   4.24862715e+00,   4.40726682e+00,
         4.54815998e+00,   4.67073930e+00,   4.77451121e+00,
         4.85905784e+00,   4.92403877e+

**TODO**: Now try to optimize the following function instead, using the same value of $\alpha$. What behavior do you observe? Note that the optimization seems to run much slower than in the previous case. Why do you think that is?

$$
\DeclareMathOperator*{\argmin}{argmin}
\argmin_{\theta_1,\ldots,\theta_n} \alpha\cdot\|\vp(\theta_1,\ldots,\theta_n)-\vp_\mathrm{target}\|_2^2 + \sum_{i=1}^n\left|\theta_i\right|
$$


## Solution 3.2

In [15]:
def solverOpti2(th, target):
    alpha = 10
    def f_obj(theta):
        return alpha*(la.norm(chain(theta) - target))**2 + np.math.fsum(np.absolute(theta))
    res = opt.minimize(f_obj, th)
    return res.x

ik_demo(solverOpti2, 5)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', min_width='125px', width='500px'), marks=[Scatter(colors=['DeepSkyBlue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.,  2.,  3.,  4.,  5.]), y=array([ 0.,  0.,  0.,  0.,  0.,  0.])), Scatter(colors=['red'], enable_move=True, interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, update_on_move=True, x=array([ 5.]), y=array([ 0.])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.,  1.,  2.,  3.,  4.,  5.]), y=array([ 0.,  0.,  0.,  0.,  0.,  0.])), Lines(colors=['gray'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=-6.0), 'y': LinearScale(max=6.0, min=-6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 5.        ,  4.98993338,  4.95977406,  4.90964349,  4.83974351,
        4.75035559,  4.64183967,  4.51463269,  4.36924689,  4.20626766,
        4.02635129,  3.83022222,  3.61867019,  3.39254706,  3.15276334,
        2.90028455,  2.63612734,  2.36135537,  2.07707507,  1.78443111,
        1.48460188,  1.17879468,  0.86824089,  0.554191  ,  0.23790958,
       -0.07932982, -0.39624978, -0.71157419, -1.02403334, -1.33236907,
       -1.63533982, -1.93172563, -2.22033306, -2.5       , -2.76960032,
       -3.02804844, -3.27430367, -3.50737444, -3.72632225, -3.93026547,
       -4.11838291, -4.28991707, -4.44417724, -4.58054229, -4.6984631 ,
       -4.79746487, -4.87714893, -4.93719444, -4.97735961, -4.99748271,
       -4.99748271, -4.97735961, -4.93719444, -4.87714893, -4.79746487,
       -4.6984631 , -4.58054229, -4.44417724, -4.28991707, -4.11838291,
       -3.93026547, -3.72632225, -3.50737444, -3.27430367, -3.02804844,
       -2.76960032, -2.5       , -2.22033306, -1.93172563, -1.63533982,
       -1.33236907, -1.02403334, -0.71157419, -0.39624978, -0.07932982,
        0.23790958,  0.554191  ,  0.86824089,  1.17879468,  1.48460188,
        1.78443111,  2.07707507,  2.36135537,  2.63612734,  2.90028455,
        3.15276334,  3.39254706,  3.61867019,  3.83022222,  4.02635129,
        4.20626766,  4.36924689,  4.51463269,  4.64183967,  4.75035559,
        4.83974351,  4.90964349,  4.95977406,  4.98993338,  5.        ]), y=array([  0.00000000e+00,   3.17119598e-01,   6.32962268e-01,
         9.46256222e-01,   1.25573994e+00,   1.56016723e+00,
         1.85831228e+00,   2.14897456e+00,   2.43098368e+00,
         2.70320409e+00,   2.96453965e+00,   3.21393805e+00,
         3.45039506e+00,   3.67295854e+00,   3.88073232e+00,
         4.07287976e+00,   4.24862715e+00,   4.40726682e+00,
         4.54815998e+00,   4.67073930e+00,   4.77451121e+00,
         4.85905784e+00,   4.92403877e+

The second optimization really runs more slowly.
I think that the second optimization is "stricter" than the first one.
In fact, the term "sum of the absolute values of the angles" is more restrictive than the term "sum of the squares of the angles" (if the values are close to zero).
Thus, in the second case, there are as many angles as possible at zero or almost zero.